In [1]:
# from katabatic.models.ganblr.models import GANBLR
# from katabatic.models.great.models import GReaT

In [2]:
# from katabatic.pipeline.train_test_split.pipeline import TrainTestSplitPipeline


In [3]:
# from katabatic.models.tabsyn.models import TabSyn

In [4]:
from utils import discretize_preprocess, encode_preprocess

In [5]:
dataset_path = "raw_data/car.csv"
output_path = "encoded_data/car.csv"

try:
    encode_preprocess(dataset_path, output_path)
except Exception as e:
    print(f"Error during preprocessing: {e}")
    raise e

Preprocessing: raw_data/car.csv
Saved preprocessed discrete dataset to: encoded_data/car.csv


In [6]:
# Diagnostic cell for kernel crash: test torch and TabSyn imports and CUDA availability
try:
    import torch
    print('torch version:', torch.__version__)
    print('CUDA available:', torch.cuda.is_available())
    from katabatic.models.tabsyn.models import TabSyn
    print('TabSyn import successful')
except Exception as e:
    print('Diagnostic import error:', e)

torch version: 2.9.0
CUDA available: False
TabSyn import successful


In [7]:
input_csv = 'encoded_data/car.csv'
output_dir = 'sample_data/car'

# Ensure all imports are available in this cell
from katabatic.pipeline.train_test_split.pipeline import TrainTestSplitPipeline
from katabatic.models.tabsyn.models import TabSyn

import torch
print('torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())

try:
    pipeline = TrainTestSplitPipeline(model=TabSyn)
    pipeline.run(input_csv=input_csv, output_dir=output_dir, synthetic_dir='synthetic/car/tabsyn', real_test_dir='sample_data/car')
except Exception as e:
    import traceback
    print('Exception:', e)
    traceback.print_exc()

torch version: 2.9.0
CUDA available: False
Loaded data with shape: (1728, 7)
Saved train/test full data
Train size: (1382, 7), Test size: (346, 7)
Train label distribution:
 6
2    0.700434
0    0.222142
1    0.039797
3    0.037627
Name: proportion, dtype: float64
Test label distribution:
 6
2    0.699422
0    0.222543
1    0.040462
3    0.037572
Name: proportion, dtype: float64
Saved X/y split
Training shape: (1382, 6) (1382,)
Test shape: (346, 6) (346,)


[TabSyn] Synthetic data saved:
  X -> synthetic/car/tabsyn/x_synth.csv
  y -> synthetic/car/tabsyn/y_synth.csv


/Users/sindhujaghosh/Library/Caches/pypoetry/virtualenvs/katabatic--g89JVPT-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sindhujaghosh/Library/Caches/pypoetry/virtualenvs/katabatic--g89JVPT-py3.11/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [20:14:19] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Results saved to: Results/car/tabsyn_tstr.csv

TSTR Evaluation Results:

LR:
Accuracy: 0.6012
F1 Score: 0.5880

MLP:
Accuracy: 0.4538
F1 Score: 0.4832

RF:
Accuracy: 0.3410
F1 Score: 0.4043

XGBoost:
Accuracy: 0.3121
F1 Score: 0.3745


In [8]:
from utils import discretize_preprocess

discretize_preprocess(
    file_path="raw_data/car.csv",
    output_path="discretized_data/car_readme.csv",  # keep a separate file to match README example
    bins=10,
    strategy="uniform",  # or 'quantile', 'kmeans'
)


Preprocessing: raw_data/car.csv
Saved preprocessed discrete dataset to: discretized_data/car_readme.csv


In [14]:
# ▶️ MANUAL GANBLR TRAIN + SAMPLE CORRECTED

import pandas as pd
import numpy as np
from katabatic.models.ganblr.models import GANBLR

# 1️⃣ Load your processed features and labels
X = pd.read_csv("sample_data/car/X_train.csv")      # Training features
y = pd.read_csv("sample_data/car/y_train.csv").values.ravel()  # Training labels

# 2️⃣ Train GANBLR
model = GANBLR()
model.fit(X, y, k=2, epochs=100, batch_size=64)

# 3️⃣ Generate synthetic samples
synthetic_data = model.sample(size=1000)  # returns features + label

print("Synthetic data shape:", synthetic_data.shape)  # Expect (1000, 7)

# 4️⃣ Split synthetic into X and y correctly
X_synth = synthetic_data[:, :-1]   # first 6 columns (features)
y_synth = synthetic_data[:, -1]    # last column (label)

# 5️⃣ Create DataFrames with correct names
X_synth_df = pd.DataFrame(X_synth, columns=X.columns)
y_synth_df = pd.Series(y_synth, name="label")  # label column name

# 6️⃣ Display preview
display(X_synth_df.head())
display(y_synth_df.head())

# 7️⃣ Save to CSV for future evaluation
X_synth_df.to_csv("synthetic/car/ganblr_manual_X.csv", index=False)
y_synth_df.to_csv("synthetic/car/ganblr_manual_y.csv", index=False)

print("Synthetic X/Y saved successfully!")


warmup run:
22/22 [==============================] - 1s 1ms/step - loss: 1.3854 - accuracy: 0.2764


Epoch 1/100: G_loss = 2.197499, G_accuracy = 0.320550, D_loss = 0.940042, D_accuracy = 0.503392


Epoch 2/100: G_loss = 1.787780, G_accuracy = 0.395080, D_loss = 0.884829, D_accuracy = 0.524650


Epoch 3/100: G_loss = 3.873619, G_accuracy = 0.453690, D_loss = 1.430511, D_accuracy = 0.503392


Epoch 4/100: G_loss = 1.447804, G_accuracy = 0.528220, D_loss = 1.331467, D_accuracy = 0.497060


...

Generating for node: 2: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 358.44it/s]

Synthetic data shape: (1000, 7)


,0,1,2,3,4,5
0,2,2,3,0,2,1
1,2,2,2,0,2,2
2,3,2,3,0,1,1
3,0,2,1,1,0,0
4,2,0,2,0,1,1


0    2
1    2
2    2
3    0
4    2
Name: label, dtype: object

Synthetic X/Y saved successfully!


In [16]:
# GReaT (light demo run)

from katabatic.models.great.models import GReaT
import pandas as pd
import torch

# Assuming you already have your preprocessed data
data = pd.read_csv("encoded_data/car.csv")

# 1️⃣ Create and train GReaT (you already ran this once; keep as-is if done)
model = GReaT(
    llm="gpt2",    # or "microsoft/DialoGPT-medium"
    epochs=1,      # keep small for testing
    batch_size=4,  # small due to CPU
)

trainer = model.fit(data)

# 2️⃣ Force CPU (avoid CUDA error)
device = "cpu"   # <- important on your Mac

# 3️⃣ Sample synthetic data on CPU
synthetic_data_great = model.sample(
    n_samples=100,
    temperature=0.7,
    device=device,     # <-- this fixes the AssertionError
)

# 4️⃣ Inspect & save
print("GReaT synthetic shape:", synthetic_data_great.shape)
display(synthetic_data_great.head())

synthetic_data_great.to_csv("synthetic/car/great_samples.csv", index=False)
print("Saved to synthetic/car/great_samples.csv")


INFO:root:Convert data into HuggingFace dataset object...
INFO:root:Create GReaT Trainer...
/Users/sindhujaghosh/Downloads/Data_bytes_ketabatics/Katabatic/katabatic/models/great/models.py:268: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `GReaTTrainer.__init__`. Use `processing_class` instead.
  great_trainer = GReaTTrainer(
INFO:root:Start training...
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
/Users/sindhujaghosh/Library/Caches/pypoetry/virtualenvs/katabatic--g89JVPT-py3.11/lib/python3.11/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)
You're using a GPT2TokenizerFast tokenizer. Please note that with a fas

Step,Training Loss


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:27<00:00,  3.70it/s]

GReaT synthetic shape: (100, 7)


,0,1,2,3,4,5,6
0,3.0,2.0,1.0,0.0,0.0,2.0,1.0
1,1.0,0.0,3.0,2.0,2.0,2.0,2.0
2,1.0,0.0,0.0,2.0,2.0,1.0,2.0
3,3.0,3.0,1.0,1.0,2.0,0.0,2.0
4,0.0,2.0,2.0,0.0,1.0,0.0,2.0


Saved to synthetic/car/great_samples.csv


In [ ]:
# Pipeline Usage

from katabatic.pipeline.train_test_split.pipeline import TrainTestSplitPipeline
from katabatic.models.ganblr.models import GANBLR

pipeline = TrainTestSplitPipeline(model=GANBLR)

results = pipeline.run(
    input_csv="discretized_data/car_readme.csv",        # or "discretized_data/car.csv"
    output_dir="sample_data/car_readme",
    synthetic_dir="synthetic/car_readme/ganblr",
    real_test_dir="sample_data/car_readme"
)

print(results)


Loaded data with shape: (1728, 7)
Saved train/test full data
Train size: (1382, 7), Test size: (346, 7)
Train label distribution:
 6
2    0.700434
0    0.222142
1    0.039797
3    0.037627
Name: proportion, dtype: float64
Test label distribution:
 6
2    0.699422
0    0.222543
1    0.040462
3    0.037572
Name: proportion, dtype: float64
Saved X/y split
Training shape: (1382, 6) (1382,)
Test shape: (346, 6) (346,)
Loaded X shape: (1382, 6), y shape: (1382,)
warmup run:
22/22 [==============================] - 1s 1ms/step - loss: 1.3710 - accuracy: 0.3213


Epoch 1/100: G_loss = 3.622290, G_accuracy = 0.387120, D_loss = 1.289678, D_accuracy = 0.497512


Epoch 2/100: G_loss = 3.749112, G_accuracy = 0.450796, D_loss = 1.329822, D_accuracy = 0.502035


Epoch 3/100: G_loss = 2.452745, G_accuracy = 0.505789, D_loss = 0.893380, D_accuracy = 0.494346


Epoch 4/100: G_loss = 4.419298, G_accuracy = 0.569465, D_loss = 1.661553, D_accuracy = 0.500226


...

Epoch 95/100: G_loss = 0.947608, G_accuracy = 0.969609, D_loss = 0.789532, D_accuracy = 0.507010


Epoch 96/100: G_loss = 0.824855, G_accuracy = 0.969609, D_loss = 0.902606, D_accuracy = 0.502940


Epoch 97/100: G_loss = 0.290849, G_accuracy = 0.969609, D_loss = 1.466328, D_accuracy = 0.508367


Epoch 98/100: G_loss = 1.614884, G_accuracy = 0.970333, D_loss = 0.956297, D_accuracy = 0.509272


Epoch 99/100: G_loss = 1.149725, G_accuracy = 0.970333, D_loss = 0.948360, D_accuracy = 0.496608


In [1]:
# TSTR Evaluation
from katabatic.evaluate.tstr.evaluation import TSTREvaluation

evaluator = TSTREvaluation(
    synthetic_dir="synthetic/car/ganblr",   # folder with synthetic CSVs
    real_test_dir="sample_data/car"         # folder with X_test, y_test
)

tstr_results = evaluator.evaluate()
print(tstr_results)


/Users/sindhujaghosh/Library/Caches/pypoetry/virtualenvs/katabatic--g89JVPT-py3.11/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:780: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/sindhujaghosh/Library/Caches/pypoetry/virtualenvs/katabatic--g89JVPT-py3.11/lib/python3.11/site-packages/xgboost/training.py:199: UserWarning: [05:39:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "scale_pos_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Results saved to: Results/car/ganblr_tstr.csv

TSTR Evaluation Results:

LR:
Accuracy: 0.7023
F1 Score: 0.6459

MLP:
Accuracy: 0.8266
F1 Score: 0.8202

RF:
Accuracy: 0.8728
F1 Score: 0.8769

XGBoost:
Accuracy: 0.8671
F1 Score: 0.8746
{'LR': {'Accuracy': 0.7023121387283237, 'F1 Score': 0.6458842075979104}, 'MLP': {'Accuracy': 0.8265895953757225, 'F1 Score': 0.8202268186439154}, 'RF': {'Accuracy': 0.8728323699421965, 'F1 Score': 0.8768939178507337}, 'XGBoost': {'Accuracy': 0.8670520231213873, 'F1 Score': 0.8746306131754542}}
